# Global Substance Abuse Mortality Analytics (1990–2024)

**Author:** Udodirim | **Portfolio Project** |  
**Dataset:** Global deaths attributable to substance abuse (drug, alcohol, tobacco) — 207 countries, 1990–2019  
**Tools:** Python · SARIMAX · Prophet · AutoTS · Power BI · Tableau

---

## Analytical Objectives

1. **Understand** historical trends in substance-related mortality across countries and continents (1990–2019)
2. **Compare** direct vs risk-attributed deaths across five cause categories
3. **Identify** the highest-burden geographies and how patterns have shifted over 30 years
4. **Forecast** mortality trajectories through 2024 using three competing time-series models
5. **Export** analysis-ready datasets for Power BI and Tableau dashboards

---

## Notebook Structure

| Section | Description |
|---|---|
| 0 | GitHub Setup |
| 1 | Environment Setup |
| 2 | Data Loading |
| 3 | Data Quality Checks |
| 4 | Data Dictionary |
| 5 | Standardisation |
| 6 | KPIs |
| 7 | EDA Visualisations |
| 8 | Feature Engineering |
| 9 | Modelling & Forecasting |
| 10 | Exports (Power BI / Tableau) |


---
## Section 0 — GitHub Repository Setup

Run this cell **once** at the start of each Colab session.  
It creates the remote repo (first time only), configures git identity, and clones/links the repo so subsequent commit cells just work.

> **Prerequisites:** Generate a GitHub Personal Access Token (PAT) with `repo` scope at  
> `GitHub → Settings → Developer Settings → Personal Access Tokens → Tokens (classic)`

In [9]:
# [0] GITHUB SETUP — create repo via API, configure git, link local folder
# Run once per session. Safe to re-run (idempotent).


import subprocess, sys
subprocess.run([sys.executable, "-m", "pip", "install", "python-dotenv"], check=True)

import os, subprocess, requests, json
from pathlib import Path

# ── USER CONFIG ───────────────────────────────────────────────────────────────
GITHUB_USERNAME  = "Udodirim"
GITHUB_EMAIL     = "udynwosu@gmail.com"
REPO_NAME        = "substance-abuse-analytics"
REPO_DESCRIPTION = "Global substance abuse mortality analytics (1990–2019) with SARIMAX/Prophet forecasting. Portfolio project — Eden Mandate AI_Tools."
REPO_PRIVATE     = False

# ── LOCAL PROJECT PATH (VSCode — update to where your data/notebook lives) ───
PROJECT_DIR = Path(r"/Users/udodirimnwosu/Desktop/Python Projects/Git-hub Project Files/substance-abuse-analytics")  # ← update this
REPO_DIR    = PROJECT_DIR  # repo lives IN your project folder
# ─────────────────────────────────────────────────────────────────────────────

# Read token from environment variable (set once in terminal — see instructions below)
from dotenv import load_dotenv
load_dotenv(override=True)
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")

HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

# 1) Create repo on GitHub if it doesn't exist yet
check = requests.get(f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}", headers=HEADERS)
if check.status_code == 200:
    print(f"✓ Repo already exists: https://github.com/{GITHUB_USERNAME}/{REPO_NAME}")
elif check.status_code == 404:
    payload = {"name": REPO_NAME, "description": REPO_DESCRIPTION,
               "private": REPO_PRIVATE, "auto_init": True}
    r = requests.post("https://api.github.com/user/repos", headers=HEADERS, json=payload)
    if r.status_code == 201:
        print(f"✓ Created repo: https://github.com/{GITHUB_USERNAME}/{REPO_NAME}")
    else:
        raise RuntimeError(f"Repo creation failed: {r.status_code} — {r.text}")
else:
    raise RuntimeError(f"GitHub API error: {check.status_code} — {check.text}")

REPO_URL = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"

# 2) Initialise git in project folder (or pull if already initialised)
git_dir = REPO_DIR / ".git"
if not git_dir.exists():
    subprocess.run(["git", "init"],                                   cwd=REPO_DIR, check=True)
    subprocess.run(["git", "remote", "add", "origin", REPO_URL],     cwd=REPO_DIR, check=True)
    subprocess.run(["git", "fetch", "origin"],                        cwd=REPO_DIR, capture_output=True)
    # Try to checkout main branch if it exists remotely
    result = subprocess.run(["git", "checkout", "-b", "main", "--track", "origin/main"],
                             cwd=REPO_DIR, capture_output=True, text=True)
    if result.returncode != 0:
        subprocess.run(["git", "checkout", "-b", "main"], cwd=REPO_DIR, capture_output=True)
    print(f"✓ Git initialised in: {REPO_DIR}")
else:
    subprocess.run(["git", "remote", "set-url", "origin", REPO_URL], cwd=REPO_DIR, capture_output=True)
    subprocess.run(["git", "pull", "origin", "main"],                 cwd=REPO_DIR, capture_output=True)
    print(f"✓ Repo already initialised — pulled latest changes.")

# 3) Configure git identity
subprocess.run(["git", "config", "user.name",  GITHUB_USERNAME], cwd=REPO_DIR)
subprocess.run(["git", "config", "user.email", GITHUB_EMAIL],    cwd=REPO_DIR)

print(f"\n✓ Git identity configured for {GITHUB_USERNAME} <{GITHUB_EMAIL}>")
print(f"✓ Working directory: {REPO_DIR}")
print("\n GitHub setup complete. Commit cells below will push to this repo.")


# ── SHARED COMMIT HELPER (used by all subsequent commit cells) ────────────────
def git_commit_push(message: str, files: list = None):
    """
    Stage, commit, and push to GitHub.
    Automatically includes the current notebook.
    Call this at the end of each major section.
    """
    # Stage specific files or everything
    if files:
        for f in files:
            subprocess.run(["git", "add", str(f)], cwd=REPO_DIR, capture_output=True)
    else:
        subprocess.run(["git", "add", "-A"], cwd=REPO_DIR, capture_output=True)

    # Commit (skip gracefully if nothing changed)
    result = subprocess.run(
        ["git", "commit", "-m", message],
        cwd=REPO_DIR, capture_output=True, text=True
    )
    if "nothing to commit" in result.stdout:
        print("ℹ  No changes to commit.")
        return

    # Push
    push = subprocess.run(
        ["git", "push", "origin", "main"],
        cwd=REPO_DIR, capture_output=True, text=True
    )
    if push.returncode == 0:
        print(f" Pushed: '{message}'")
        print(f"   → https://github.com/{GITHUB_USERNAME}/{REPO_NAME}")
    else:
        print(" Push failed:", push.stderr)
        


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


✓ Repo already exists: https://github.com/Udodirim/substance-abuse-analytics
✓ Repo already initialised — pulled latest changes.

✓ Git identity configured for Udodirim <udynwosu@gmail.com>
✓ Working directory: /Users/udodirimnwosu/Desktop/Python Projects/Git-hub Project Files/substance-abuse-analytics

 GitHub setup complete. Commit cells below will push to this repo.


---
## Section 1 — Environment Setup

In [1]:
# [1] ENVIRONMENT SETUP — make folders, install libs, set global options

import subprocess, sys

# ── Install dependencies ───────────────────────────────────────────────────────
subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "pandas", "numpy", "matplotlib", "tqdm",
    "statsmodels", "pyarrow", "prophet", "autots"
], check=True)

from pathlib import Path
import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from tqdm import tqdm

# ── Project directories (VSCode — local path) ─────────────────────────────────
PROJECT_DIR = Path("/Users/udodirimnwosu/Desktop/Python Projects/Git-hub Project Files/substance-abuse-analytics")
EXPORTS_DIR = PROJECT_DIR / "exports"
FIG_DIR     = EXPORTS_DIR / "figures"
for d in (PROJECT_DIR, EXPORTS_DIR, FIG_DIR):
    d.mkdir(parents=True, exist_ok=True)

# ── Global settings ────────────────────────────────────────────────────────────
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
np.random.seed(42); random.seed(42)
pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", "{:,.2f}".format)
plt.style.use("ggplot")

# ── Optional library availability flags ───────────────────────────────────────
PROPHET_OK = True
AUTOTS_OK  = True
try:
    from prophet import Prophet
    import logging
    logging.getLogger("prophet").setLevel(logging.WARNING)
    logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
except Exception as e:
    PROPHET_OK = False
    print(f"Prophet not available: {e}")
try:
    from autots import AutoTS
except Exception as e:
    AUTOTS_OK = False
    print(f"AutoTS not available: {e}")

print(f"Setup complete ✓  |  Prophet: {PROPHET_OK}  |  AutoTS: {AUTOTS_OK}")
print(f"Project dir : {PROJECT_DIR}")
print(f"Exports dir : {EXPORTS_DIR}")
print(f"Figures dir : {FIG_DIR}")


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
/Users/udodirimnwosu/Desktop/Python Projects/Git-hub Project Files/substance-abuse-analytics/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


Setup complete ✓  |  Prophet: True  |  AutoTS: True
Project dir : /Users/udodirimnwosu/Desktop/Python Projects/Git-hub Project Files/substance-abuse-analytics
Exports dir : /Users/udodirimnwosu/Desktop/Python Projects/Git-hub Project Files/substance-abuse-analytics/exports
Figures dir : /Users/udodirimnwosu/Desktop/Python Projects/Git-hub Project Files/substance-abuse-analytics/exports/figures


---
## Section 2 — Data Loading

In [3]:
# [2] LOAD DATA — read CSV from local folder and quick sanity printouts

import os, numpy as np, pandas as pd
from pathlib import Path

# ── Update this to where your CSV file is saved locally ───────────────────────
RAW_CSV_PATH = PROJECT_DIR / "_abuse.csv"  # ← put your CSV inside your project folder
# ─────────────────────────────────────────────────────────────────────────────

assert os.path.exists(RAW_CSV_PATH), (
    f"CSV not found at: {RAW_CSV_PATH}\n"
    f"Make sure _abuse.csv is inside: {PROJECT_DIR}"
)

try:
    df = pd.read_csv(RAW_CSV_PATH, engine="pyarrow")
except Exception:
    df = pd.read_csv(RAW_CSV_PATH)

print("Loaded :", RAW_CSV_PATH)
print("Shape  :", df.shape)
print(f"Memory : {df.memory_usage(deep=True).sum() / 1e6:.3f} MB")
print("\nColumns:", df.columns.tolist())
display(df.head(5))
print("\nDtypes:")
print(df.dtypes)

# Auto-detect time axis
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    print(f"\n'date' column detected. Range: {df['date'].min()} → {df['date'].max()}")
elif "year" in df.columns or "Year" in df.columns:
    yr = "year" if "year" in df.columns else "Year"
    df[yr] = pd.to_numeric(df[yr], errors="coerce")
    print(f"\n'{yr}' column detected. Range: {int(df[yr].min())} → {int(df[yr].max())}")
else:
    print("\nNo 'date' or 'year' column detected.")

Loaded : /Users/udodirimnwosu/Desktop/Python Projects/Git-hub Project Files/substance-abuse-analytics/_abuse.csv
Shape  : (6210, 8)
Memory : 0.494 MB

Columns: ['continent', 'country', 'year', 'deaths_direct_drug', 'deaths_direct_alcohol', 'deaths_risk_tobacco', 'deaths_risk_drug', 'deaths_risk_alcohol']


,continent,country,year,deaths_direct_drug,deaths_direct_alcohol,deaths_risk_tobacco,deaths_risk_drug,deaths_risk_alcohol
0,Asia,Afghanistan,1990,93,72,9723,174,356
1,Europe,Albania,1990,5,10,3468,64,289
2,Africa,Algeria,1990,171,75,17985,296,496
3,Oceania,American Samoa,1990,0,0,39,1,3
4,Europe,Andorra,1990,0,1,62,2,22



Dtypes:
continent                  str
country                    str
year                     int64
deaths_direct_drug       int64
deaths_direct_alcohol    int64
deaths_risk_tobacco      int64
deaths_risk_drug         int64
deaths_risk_alcohol      int64
dtype: object

'year' column detected. Range: 1990 → 2019


---
## Section 3 — Data Quality Checks

In [5]:
# [3] DATA QUALITY CHECKS — nulls, empties, duplicates, negative values

import pandas as pd, numpy as np

def strip_object_cols(df_):
    for c in df_.select_dtypes(include="str").columns:
        df_[c] = df_[c].astype(str).str.strip()
    return df_

df = strip_object_cols(df)

# Nulls
null_by_col  = df.isna().sum()
total_nulls  = int(null_by_col.sum())
print(f"Total nulls: {total_nulls}")
if total_nulls:
    display(null_by_col[null_by_col > 0].sort_values(ascending=False))
else:
    print("  ✓ No nulls detected")

# Empty strings
empty_like = (df.select_dtypes(include="str") == "").sum()
if (empty_like > 0).any():
    print("\nEmpty-string counts:"); display(empty_like[empty_like > 0])
else:
    print("  ✓ No empty-string cells")

# Duplicates
dup_count = int(df.duplicated().sum())
print(f"\nExact duplicate rows: {dup_count}")
if dup_count > 0:
    df = df.drop_duplicates().reset_index(drop=True)
    print(f"  Dropped {dup_count} rows. New shape: {df.shape}")
else:
    print("  ✓ No duplicate rows")

# Negative death values (impossible)
num_cols   = df.select_dtypes(include=[np.number]).columns.tolist()
neg_counts = (df[num_cols] < 0).sum()
if (neg_counts > 0).any():
    print("\n⚠ Negative values found:"); display(neg_counts[neg_counts > 0])
else:
    print("  ✓ No negative values in numeric columns")

print(f"\nFinal shape: {df.shape}")


Total nulls: 0
  ✓ No nulls detected
  ✓ No empty-string cells

Exact duplicate rows: 0
  ✓ No duplicate rows
  ✓ No negative values in numeric columns

Final shape: (6210, 8)


---
## Section 4 — Data Dictionary

In [6]:
# [4] DATA DICTIONARY — schema overview with sample values

import pandas as pd

def data_dictionary(df_: pd.DataFrame, sample_n: int = 3) -> pd.DataFrame:
    rows = []
    for col in df_.columns:
        s = df_[col]
        rows.append({
            "column"       : col,
            "dtype"        : str(s.dtype),
            "missing_count": int(s.isna().sum()),
            "missing_pct"  : round(100 * s.isna().mean(), 2),
            "n_unique"     : int(s.nunique(dropna=True)),
            "sample_values": "; ".join(map(lambda x: str(x)[:50], s.dropna().unique()[:sample_n])),
        })
    return pd.DataFrame(rows).sort_values("column").reset_index(drop=True)

dict_df = data_dictionary(df, sample_n=3)
display(dict_df)


,column,dtype,missing_count,missing_pct,n_unique,sample_values
0,continent,str,0,0.00,6,Asia; Europe; Africa
1,country,str,0,0.00,207,Afghanistan; Albania; Algeria
2,deaths_direct_alcohol,int64,0,0.00,1288,72; 10; 75
3,deaths_direct_drug,int64,0,0.00,882,93; 5; 171
4,deaths_risk_alcohol,int64,0,0.00,3815,356; 289; 496
5,deaths_risk_drug,int64,0,0.00,1979,174; 64; 296
6,deaths_risk_tobacco,int64,0,0.00,4764,9723; 3468; 17985
7,year,int64,0,0.00,30,1990; 1991; 1992


---
## Section 5 — Standardisation & Shared Utilities

All shared constants, colour maps, and helper functions are defined **once here** and referenced throughout the notebook.

In [7]:
# [5] STANDARDISATION + SHARED UTILITIES
# ─────────────────────────────────────────────────────────────────────────────
# Everything defined here is used by all downstream cells.
# DO NOT redefine MEASURES / PRETTY / MEASURE_COLORS elsewhere.
# ─────────────────────────────────────────────────────────────────────────────

import pandas as pd
import numpy as np
from pathlib import Path

# ── 5.1 Column normalisation ───────────────────────────────────────────────────
def to_snake(s: str) -> str:
    return s.replace("/"," ").replace("-"," ").replace("("," ").replace(")"," ").replace(":"," ")

df.columns = (df.columns.map(str).map(to_snake)
               .str.lower().str.replace(r"[^a-z0-9]+", "_", regex=True).str.strip("_"))
df = df.rename(columns={"entity": "country", "code": "iso3"})

# ── 5.2 Date handling ─────────────────────────────────────────────────────────
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
elif "year" in df.columns:
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    df["date"] = pd.to_datetime(df["year"].astype("string") + "-01-01", errors="coerce")

# ── 5.3 Numeric coercion ──────────────────────────────────────────────────────
DIM_COLS = {"country", "continent", "region", "iso3", "date", "year"}
for c in df.columns:
    if c not in DIM_COLS and df[c].dtype == "object":
        df[c] = pd.to_numeric(df[c], errors="coerce")

# ── 5.4 Tidy categoricals ─────────────────────────────────────────────────────
for c in ["country", "continent", "region", "iso3"]:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()

# ── 5.5 Derived column: direct total ──────────────────────────────────────────
if {"deaths_direct_drug", "deaths_direct_alcohol"}.issubset(df.columns):
    df["deaths_direct_total"] = df["deaths_direct_drug"] + df["deaths_direct_alcohol"]

# ── 5.6 SHARED CONSTANTS ──────────────────────────────────────────────────────
MEASURES = [
    "deaths_direct_drug",
    "deaths_direct_alcohol",
    "deaths_risk_tobacco",
    "deaths_risk_drug",
    "deaths_risk_alcohol",
]
PRETTY = {
    "deaths_direct_drug"   : "Direct: Drug",
    "deaths_direct_alcohol": "Direct: Alcohol",
    "deaths_risk_tobacco"  : "Risk: Tobacco",
    "deaths_risk_drug"     : "Risk: Drug",
    "deaths_risk_alcohol"  : "Risk: Alcohol",
    "deaths_direct_total"  : "Direct Total (Drug + Alcohol)",
}
MEASURE_COLORS = {
    "deaths_direct_drug"   : "#e74c3c",
    "deaths_direct_alcohol": "#3498db",
    "deaths_risk_tobacco"  : "#2ecc71",
    "deaths_risk_drug"     : "#f39c12",
    "deaths_risk_alcohol"  : "#8e44ad",
    "deaths_direct_total"  : "#1abc9c",
}
PRIMARY_MEASURE = "deaths_direct_total"

# ── 5.7 SHARED HELPER FUNCTIONS ───────────────────────────────────────────────

def safe_save_csv(df_: pd.DataFrame, path, overwrite: bool = True) -> Path:
    """Save DataFrame to CSV. Overwrites by default for clean re-runs."""
    path = Path(path)
    df_.to_csv(path, index=False)
    print(f"  Saved → {path.name}")
    return path


def safe_save_fig(fig, path) -> Path:
    """Save figure. Overwrites by default."""
    path = Path(path)
    fig.savefig(path, dpi=150, bbox_inches="tight")
    plt.close(fig)
    print(f"  Saved → {path.name}")
    return path


def bar_palette(n: int):
    """Return n distinct colours from tab20."""
    return [plt.cm.get_cmap("tab20")(i) for i in np.linspace(0, 1, max(n, 2))]


def top_n_all_years(dfx: pd.DataFrame, level: str, measure: str, n: int = 5) -> pd.DataFrame:
    """Sum a measure over all years, return top N entities with share %."""
    out = (dfx.dropna(subset=[level])
              .groupby(level, as_index=False)[measure].sum()
              .sort_values(measure, ascending=False).head(n))
    out["share_pct"] = out[measure] / dfx[measure].sum() * 100
    return out.rename(columns={level: "name", measure: "value"}).reset_index(drop=True)


def regression_metrics(y_true, y_pred) -> dict:
    """Return MAE, RMSE, and MAPE."""
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    mae  = float(np.mean(np.abs(y_true - y_pred)))
    rmse = float(np.sqrt(np.mean((y_true - y_pred) ** 2)))
    mape = float(np.mean(np.abs((y_true - y_pred) / np.maximum(1e-9, np.abs(y_true)))) * 100)
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape}


# Verify
print("Columns :", df.columns.tolist())
print("Date range:", df['date'].min().year, "→", df['date'].max().year)
print("\nShared constants ready:")
print(f"  MEASURES ({len(MEASURES)}):", MEASURES)
print(f"  PRIMARY_MEASURE: {PRIMARY_MEASURE}")


Columns : ['continent', 'country', 'year', 'deaths_direct_drug', 'deaths_direct_alcohol', 'deaths_risk_tobacco', 'deaths_risk_drug', 'deaths_risk_alcohol', 'date', 'deaths_direct_total']
Date range: 1990 → 2019

Shared constants ready:
  MEASURES (5): ['deaths_direct_drug', 'deaths_direct_alcohol', 'deaths_risk_tobacco', 'deaths_risk_drug', 'deaths_risk_alcohol']
  PRIMARY_MEASURE: deaths_direct_total


In [10]:
# ── COMMIT 1: Data loading, quality checks, standardisation ──────────────────
git_commit_push("feat: data loading, quality checks, and standardisation (Sections 1–5)")


 Pushed: 'feat: data loading, quality checks, and standardisation (Sections 1–5)'
   → https://github.com/Udodirim/substance-abuse-analytics


---
## Section 6 — Key Performance Indicators (KPIs)

In [ ]:
# [6] KPIs — global totals, YoY%, 3-yr rolling avg, per measure

import pandas as pd, numpy as np

def build_kpis(df_: pd.DataFrame, measures: list) -> pd.DataFrame:
    annual = df_.groupby("date", as_index=True)[measures].sum().sort_index()
    rows = []
    for m in measures:
        s = annual[m].dropna()
        tot   = float(s.sum())
        last  = float(s.iloc[-1]) if len(s) else np.nan
        yoy   = float((s.iloc[-1] - s.iloc[-2]) / s.iloc[-2] * 100) if len(s) >= 2 and s.iloc[-2] != 0 else np.nan
        roll3 = float(s.rolling(3).mean().iloc[-1]) if len(s) >= 3 else np.nan
        rows.append({
            "measure"               : PRETTY.get(m, m),
            "period"                : f"{s.index.min().year}–{s.index.max().year}",
            "total_all_years"       : tot,
            "last_year"             : s.index[-1].year,
            "last_year_value"       : last,
            "yoy_change_pct"        : round(yoy, 2) if not np.isnan(yoy) else None,
            "3yr_rolling_avg"       : roll3,
        })
    return pd.DataFrame(rows)

ALL_MEASURES = MEASURES + (["deaths_direct_total"] if "deaths_direct_total" in df.columns else [])
kpi_df = build_kpis(df, ALL_MEASURES)
display(kpi_df.style.format({
    "total_all_years": "{:,.0f}",
    "last_year_value": "{:,.0f}",
    "3yr_rolling_avg": "{:,.0f}",
    "yoy_change_pct" : "{:.2f}%",
}))

safe_save_csv(kpi_df, PROJECT_DIR / "kpi_all.csv")


---
## Section 7 — EDA Visualisations

Four key charts:  
1. Global trend (all measures)  
2. Top 15 countries in the final year  
3. Top 5 countries & continents per measure (sum 1990–2019)  
4. Composition pie + bar (full period)

In [ ]:
# [7A] GLOBAL TREND — all measures on one chart

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

annual = (df.groupby("date", as_index=False)[MEASURES].sum()
            .sort_values("date").reset_index(drop=True))
annual["year"] = annual["date"].dt.year

fig, ax = plt.subplots(figsize=(12, 5))
for m in MEASURES:
    ax.plot(annual["date"], annual[m], marker=".", linewidth=1.5,
            label=PRETTY[m], color=MEASURE_COLORS[m])
ax.set_title("Global Substance Abuse Mortality Trend (1990–2019)", fontsize=13, fontweight="bold")
ax.set_xlabel("Year")
ax.set_ylabel("Deaths")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x/1e6:.1f}M"))
ax.legend(loc="upper left", framealpha=0.8)
plt.tight_layout()
plt.show()
safe_save_fig(fig, FIG_DIR / "01_trend_global_all_measures.png")


In [ ]:
# [7B] TOP 15 COUNTRIES — last year in dataset

EXCLUDE = {"World","High-income","Upper-middle-income","Lower-middle-income","Low-income"}
df_c   = df[~df["country"].isin(EXCLUDE)].copy()
last_yr = int(df["date"].dt.year.max())

last_df = df_c[df_c["date"].dt.year == last_yr]
by_cty  = (last_df.groupby("country", as_index=False)["deaths_direct_total"]
                   .sum()
                   .sort_values("deaths_direct_total", ascending=False)
                   .head(15))

fig, ax = plt.subplots(figsize=(11, 6))
colors  = bar_palette(len(by_cty))
ax.barh(by_cty["country"][::-1].values, by_cty["deaths_direct_total"][::-1].values, color=colors)
for i, (val, row) in enumerate(zip(by_cty["deaths_direct_total"][::-1], by_cty[::-1].itertuples())):
    ax.text(val, i, f"  {val:,.0f}", va="center", fontsize=8)
ax.set_title(f"Top 15 Countries — Direct Substance Deaths ({last_yr})", fontsize=13, fontweight="bold")
ax.set_xlabel("Deaths")
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x/1e3:.0f}K"))
plt.tight_layout()
plt.show()
safe_save_fig(fig, FIG_DIR / f"02_top15_countries_{last_yr}.png")


In [ ]:
# [7C] TOP 5 COUNTRIES & CONTINENTS PER MEASURE (sum 1990–2019)

for m in MEASURES:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f"{PRETTY[m]} — Sum 1990–2019", fontsize=12, fontweight="bold")

    for ax, level in zip(axes, ["country", "continent"]):
        if level not in df.columns:
            ax.axis("off"); ax.set_title(f"No '{level}' column"); continue
        t = top_n_all_years(df_c if level == "country" else df, level, m, n=5)
        ax.barh(t["name"][::-1], t["value"][::-1], color=bar_palette(5))
        for i, (val, pc) in enumerate(zip(t["value"][::-1], t["share_pct"][::-1])):
            ax.text(val, i, f"  {val:,.0f} ({pc:.1f}%)", va="center", fontsize=8)
        ax.set_title(f"Top 5 {level.title()}s")
        ax.set_xlabel("Deaths")
        ax.grid(axis="x", alpha=0.3)

    plt.tight_layout()
    plt.show()
    safe_save_fig(fig, FIG_DIR / f"03_top5_{m}.png")


In [ ]:
# [7D] COMPOSITION — pie + bar (full 1990–2019 period)

comp   = df[MEASURES].sum(numeric_only=True)
labels = [m for m in MEASURES if comp.get(m, 0) > 0]
vals   = np.array([float(comp[m]) for m in labels])
pct    = 100 * vals / vals.sum()

wedge_colors  = [MEASURE_COLORS[l] for l in labels]
explode       = [0.06 if p < 3 else 0 for p in pct]
legend_labels = [f"{PRETTY.get(l, l)} — {p:.1f}%" for l, p in zip(labels, pct)]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle("Mortality Composition by Cause — Sum 1990–2019", fontsize=13, fontweight="bold")

# Pie
wedges, _ = axes[0].pie(vals, labels=None, autopct=None,
                        explode=explode, startangle=90, colors=wedge_colors)
axes[0].axis("equal")
axes[0].legend(wedges, legend_labels, title="Cause",
               loc="center left", bbox_to_anchor=(1.0, 0.5), frameon=False, fontsize=9)

# Horizontal bar
order    = np.argsort(vals)
bar_lbls = [PRETTY.get(labels[i], labels[i]) for i in order]
bar_cols = [MEASURE_COLORS[labels[i]] for i in order]
axes[1].barh(bar_lbls, vals[order], color=bar_cols)
for i, (v, p) in enumerate(zip(vals[order], pct[order])):
    axes[1].text(v, i, f"  {v:,.0f}  ({p:.1f}%)", va="center", fontsize=8)
axes[1].set_xlabel("Deaths")
axes[1].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x/1e9:.1f}B"))
axes[1].grid(axis="x", alpha=0.3)

plt.tight_layout()
plt.show()
safe_save_fig(fig, FIG_DIR / "04_composition_1990_2019.png")


In [ ]:
# ── COMMIT 2: KPIs and EDA visualisations ────────────────────────────────────
git_commit_push("feat: KPIs and EDA visualisations (Sections 6–7)")


---
## Section 8 — Feature Engineering

In [ ]:
# [8] FEATURE ENGINEERING — annual global series + lag/rolling features for modelling

import pandas as pd, numpy as np

# Global annual series for primary measure
y = (df.groupby("date", as_index=True)[PRIMARY_MEASURE]
        .sum()
        .sort_index())
y.name = "y"

# Feature frame
fe = y.to_frame().copy()
fe["year"]  = fe.index.year
fe["lag1"]  = fe["y"].shift(1)
fe["lag2"]  = fe["y"].shift(2)
fe["roll3"] = fe["y"].rolling(3, min_periods=3).mean()   # strict: NaN until 3rd obs
fe["trend"] = np.arange(len(fe))                          # linear time index

fe_model = fe.dropna(subset=["lag1", "lag2", "roll3"]).copy()

print(f"Full series : {len(fe)} years ({fe.index.min().year}–{fe.index.max().year})")
print(f"Model-ready : {len(fe_model)} years (dropped {len(fe)-len(fe_model)} for warm-up)")
print("\nFirst 5 rows:")
display(fe_model.head())
print("\nLast 5 rows:")
display(fe_model.tail())


In [ ]:
# ── COMMIT 3: Feature engineering ────────────────────────────────────────────
git_commit_push("feat: feature engineering with lag and rolling window features (Section 8)")


---
## Section 9 — Modelling & Forecasting

Three candidate models are trained and evaluated on a 3-year hold-out test set:  
- **SARIMAX** — grid search over (p,d,q) orders, selected by AIC  
- **Prophet** — Facebook's additive decomposition model (configured correctly for annual data)  
- **AutoTS** — automated ensemble selector  

Winner is chosen by lowest RMSE, then refitted on the full series to produce a 5-year forecast.

In [ ]:
# [9A] MODEL TRAINING & EVALUATION — SARIMAX vs Prophet vs AutoTS

import json, numpy as np, pandas as pd, warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
warnings.filterwarnings("ignore")

H          = 5   # forecast horizon (years)
TEST_YEARS = 3   # hold-out period

def split_series(series, test_periods=3):
    series = series.dropna()
    test_periods = min(test_periods, max(1, len(series) // 4))
    return series.iloc[:-test_periods], series.iloc[-test_periods:]

y_train, y_test = split_series(y, TEST_YEARS)
print(f"Train: {y_train.index.min().year}–{y_train.index.max().year}  ({len(y_train)} obs)")
print(f"Test : {y_test.index.min().year}–{y_test.index.max().year}   ({len(y_test)} obs)")

candidates = []

# ── SARIMAX: grid over (p,d,q) ────────────────────────────────────────────────
print("\n[1/3] Fitting SARIMAX grid...")
best_sarimax = (None, np.inf, None)
for p in (0, 1, 2):
    for d in (0, 1):
        for q in (0, 1, 2):
            try:
                fit = SARIMAX(y_train, order=(p, d, q),
                              enforce_stationarity=False,
                              enforce_invertibility=False).fit(disp=False)
                if fit.aic < best_sarimax[1]:
                    best_sarimax = ((p, d, q), fit.aic, fit)
            except Exception:
                pass

sarimax_order, sarimax_aic, sarimax_fit = best_sarimax
if sarimax_fit:
    pred = sarimax_fit.get_forecast(steps=len(y_test)).predicted_mean
    m    = regression_metrics(y_test.values, pred.values)
    candidates.append({"model": "SARIMAX", "params": {"order": sarimax_order, "AIC": round(sarimax_aic, 2)}, "metrics": m})
    print(f"  Best order: {sarimax_order}  AIC: {sarimax_aic:.2f}  MAPE: {m['MAPE']:.2f}%")

# ── Prophet: configured for annual data ──────────────────────────────────────
# NOTE: with annual (YS) frequency, sub-annual seasonalities are meaningless.
# yearly_seasonality is also disabled — 1 obs/year means no intra-year cycle to fit.
if PROPHET_OK:
    print("\n[2/3] Fitting Prophet...")
    try:
        df_tr = y_train.reset_index().rename(columns={"date": "ds", PRIMARY_MEASURE: "y"})
        m_prophet = Prophet(
            seasonality_mode="additive",
            yearly_seasonality=False,    # ← correct for annual data
            weekly_seasonality=False,
            daily_seasonality=False,
            changepoint_prior_scale=0.3,
            n_changepoints=min(10, len(df_tr) - 2)  # ← guard against n_changepoints > n_obs
        )
        m_prophet.fit(df_tr)
        future = pd.DataFrame({"ds": y_test.index})
        fc     = m_prophet.predict(future)
        m      = regression_metrics(y_test.values, fc["yhat"].values)
        candidates.append({"model": "Prophet", "params": {"changepoint_prior": 0.3}, "metrics": m})
        print(f"  MAPE: {m['MAPE']:.2f}%")
    except Exception as e:
        PROPHET_OK = False
        print(f"  Prophet failed: {e}")

# ── AutoTS ────────────────────────────────────────────────────────────────────
if AUTOTS_OK:
    print("\n[3/3] Fitting AutoTS...")
    try:
        at = AutoTS(
            forecast_length=len(y_test), frequency="infer",
            model_list="fast", transformer_list="fast",
            ensemble="simple", max_generations=3,
            num_validations=2, random_seed=42, verbose=0
        ).fit(df=y_train.to_frame(name="value"),
              date_col=y_train.index.name, value_col="value", id_col=None)
        autots_pred = at.predict().forecast["value"].values
        m = regression_metrics(y_test.values, autots_pred)
        candidates.append({"model": "AutoTS(fast)", "params": "auto", "metrics": m})
        print(f"  MAPE: {m['MAPE']:.2f}%")
    except Exception as e:
        AUTOTS_OK = False
        print(f"  AutoTS failed: {e}")

print("\nDone.")


In [ ]:
# [9B] MODEL COMPARISON TABLE — clear display of all candidates

import pandas as pd, numpy as np

def choose_best(cands: list) -> dict:
    valid = [c for c in cands if c["metrics"] is not None]
    return min(valid, key=lambda c: (c["metrics"]["RMSE"], c["metrics"]["MAE"])) if valid else None

winner = choose_best(candidates)

# Build display table
rows = []
for c in candidates:
    if c["metrics"]:
        rows.append({
            "Model"   : c["model"],
            "MAE"     : c["metrics"]["MAE"],
            "RMSE"    : c["metrics"]["RMSE"],
            "MAPE (%)" : round(c["metrics"]["MAPE"], 2),
            "Winner"  : "✅" if winner and c["model"] == winner["model"] else ""
        })

eval_df = pd.DataFrame(rows).sort_values("RMSE").reset_index(drop=True)

print("=" * 55)
print("  MODEL EVALUATION SUMMARY (hold-out test set)")
print(f"  Test period: {y_test.index.min().year}–{y_test.index.max().year}  ({len(y_test)} years)")
print("=" * 55)
display(eval_df.style
    .format({"MAE": "{:,.0f}", "RMSE": "{:,.0f}", "MAPE (%)": "{:.2f}"})
    .highlight_min(subset=["RMSE"], color="#d4edda")
)

if winner:
    print(f"\n🏆 Winner: {winner['model']}")
    print(f"   MAPE = {winner['metrics']['MAPE']:.2f}%  |  "
          f"RMSE = {winner['metrics']['RMSE']:,.0f}  |  "
          f"MAE = {winner['metrics']['MAE']:,.0f}")
else:
    print("No valid model fitted. Check library installations.")

# Save eval to CSV
safe_save_csv(eval_df, PROJECT_DIR / "model_evaluation.csv")

# Save full report JSON
report = {"primary_measure": PRIMARY_MEASURE, "test_years": TEST_YEARS,
          "candidates": candidates, "winner": winner}
with open(PROJECT_DIR / "model_report.json", "w") as f:
    json.dump(report, f, indent=2, default=str)
print("  Saved → model_report.json")


In [ ]:
# [9C] MULTI-MEASURE FORECASTS — SARIMAX per measure, 5-year horizon

import numpy as np, pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

year_max   = int(df["date"].dt.year.max())
future_idx = pd.date_range(
    start=pd.Timestamp(year_max, 1, 1) + pd.offsets.YearBegin(1),
    periods=H, freq="YS"
)

def best_sarimax_fit(series: pd.Series):
    """AIC grid search over (p,d,q); returns (order, fitted_model)."""
    series = series.dropna()
    best   = (None, np.inf, None)
    for p in (0, 1, 2):
        for d in (0, 1):
            for q in (0, 1, 2):
                try:
                    fit = SARIMAX(series, order=(p, d, q),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False).fit(disp=False)
                    if fit.aic < best[1]:
                        best = ((p, d, q), fit.aic, fit)
                except Exception:
                    pass
    if best[2] is None:
        fit = SARIMAX(series, order=(0, 1, 1),
                      enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
        return (0, 1, 1), fit
    return best[0], best[2]

all_forecasts = []
order_report  = []

for m in MEASURES:
    y_m = df.groupby("date", as_index=True)[m].sum().sort_index()
    if y_m.fillna(0).sum() == 0 or y_m.dropna().shape[0] < 8:
        print(f"Skipping {m}: insufficient data.")
        continue

    order, fit = best_sarimax_fit(y_m)
    f     = fit.get_forecast(steps=H)
    fmean = f.predicted_mean
    conf  = f.conf_int(alpha=0.2)

    fc = pd.DataFrame({
        "date"      : future_idx[:len(fmean)],
        "measure"   : m,
        "prediction": fmean.values[:len(future_idx)],
        "lower"     : conf.iloc[:, 0].values[:len(future_idx)],
        "upper"     : conf.iloc[:, 1].values[:len(future_idx)],
    })
    all_forecasts.append(fc)
    order_report.append({"measure": m, "sarimax_order": order})
    print(f"  {PRETTY[m]:<30} order={order}")

forecast_long = pd.concat(all_forecasts, ignore_index=True).sort_values(["date", "measure"])
forecast_wide = (forecast_long.pivot(index="date", columns="measure", values="prediction")
                 .reset_index().sort_values("date"))

print("\nForecast preview:")
display(forecast_wide)

safe_save_csv(forecast_long, PROJECT_DIR / "forecast_by_measure_long.csv")
safe_save_csv(forecast_wide, PROJECT_DIR / "forecast_by_measure_wide.csv")
with open(PROJECT_DIR / "model_report_by_measure.json", "w") as f:
    json.dump(order_report, f, indent=2)
print("  Saved → model_report_by_measure.json")


In [ ]:
# [9D] FORECAST VISUALISATION — actual vs forecast with confidence intervals

import matplotlib.pyplot as plt, matplotlib.ticker as mticker

actuals_long = (df.groupby("date", as_index=False)[MEASURES].sum()
                  .melt(id_vars="date", var_name="measure", value_name="actual"))

combo = pd.merge(actuals_long, forecast_long, on=["date", "measure"], how="outer").sort_values(["measure", "date"])

for m in MEASURES:
    sub = combo[combo["measure"] == m].sort_values("date")
    fig, ax = plt.subplots(figsize=(11, 4))

    a = sub.dropna(subset=["actual"])
    f = sub.dropna(subset=["prediction"])

    if not a.empty:
        ax.plot(a["date"], a["actual"], marker="o", lw=1.5, label="Actual",
                color=MEASURE_COLORS[m])
    if not f.empty:
        ax.fill_between(f["date"], f["lower"], f["upper"],
                        alpha=0.15, color=MEASURE_COLORS[m], label="80% CI")
        ax.plot(f["date"], f["prediction"], marker="o", ls="--", lw=1.5,
                color=MEASURE_COLORS[m], label="Forecast")

    # Shade forecast region
    if not f.empty:
        ax.axvline(x=a["date"].max(), color="grey", ls=":", lw=0.8)

    ax.set_title(f"Actual vs Forecast — {PRETTY[m]}", fontsize=11, fontweight="bold")
    ax.set_xlabel("Year")
    ax.set_ylabel("Deaths")
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x/1e6:.2f}M"))
    ax.legend()
    plt.tight_layout()
    plt.show()
    safe_save_fig(fig, FIG_DIR / f"05_forecast_{m}.png")


In [ ]:
# ── COMMIT 4: Modelling and forecasting ──────────────────────────────────────
git_commit_push("feat: SARIMAX/Prophet/AutoTS modelling, evaluation, and 5-year forecasts (Section 9)")


---
## Section 10 — Exports

All output files are saved in flat CSVs suitable for both **Power BI** and **Tableau**.

| File | Description |
|---|---|
| `annual_totals.csv` | One row per year, global totals |
| `country_year.csv` | Country × year with computed totals |
| `measures_long.csv` | Annual data in long/tidy format |
| `fact_actual_forecast.csv` | Actuals + forecasts combined |
| `kpi_all.csv` | KPI summary table |
| `model_evaluation.csv` | Model comparison metrics |

In [ ]:
# [10A] EXPORT — annual totals, country-year, long format

import pandas as pd

ALL_M = MEASURES + (["deaths_direct_total"] if "deaths_direct_total" in df.columns else [])

# Annual global totals
annual = (df.groupby("date", as_index=False)[ALL_M].sum()
            .sort_values("date").reset_index(drop=True))
annual["year"] = annual["date"].dt.year
safe_save_csv(annual[["date", "year"] + ALL_M], PROJECT_DIR / "annual_totals.csv")

# Country × year
group_cols  = [c for c in ["continent", "country", "date"] if c in df.columns]
country_year = df.groupby(group_cols, as_index=False)[MEASURES].sum().sort_values(group_cols)
country_year["deaths_direct_total"] = country_year["deaths_direct_drug"] + country_year["deaths_direct_alcohol"]
country_year["year"]                = pd.to_datetime(country_year["date"]).dt.year
safe_save_csv(country_year, PROJECT_DIR / "country_year.csv")

# Long / tidy format
long_df = annual.melt(id_vars=["date", "year"], value_vars=ALL_M,
                      var_name="measure", value_name="value")
long_df["measure_label"] = long_df["measure"].map(PRETTY)
safe_save_csv(long_df, PROJECT_DIR / "measures_long.csv")

print("\n✓ Core export files saved.")


In [ ]:
# [10B] FACT TABLE — actuals + forecasts combined (Power BI / Tableau ready)

import pandas as pd

actuals_long = annual.melt(id_vars=["date", "year"], value_vars=MEASURES,
                            var_name="measure", value_name="actual")

combo = pd.merge(actuals_long, forecast_long, on=["date", "measure"], how="outer")
combo["year"]        = combo["year"].fillna(combo["date"].dt.year).astype(int)
combo["is_forecast"] = combo["prediction"].notna() & combo["actual"].isna()
combo["data_type"]   = combo["is_forecast"].map({True: "Forecast", False: "Actual"})
combo["measure_label"] = combo["measure"].map(PRETTY)
combo = combo.sort_values(["measure", "date"]).reset_index(drop=True)

display(combo.head(10))
safe_save_csv(combo, PROJECT_DIR / "fact_actual_forecast.csv")

print(f"\nFact table: {len(combo)} rows, {combo['measure'].nunique()} measures")
print(f"Years covered: {combo['year'].min()}–{combo['year'].max()}")


In [ ]:
# [10C] ENHANCED COUNTRY TABLE — decade grouping, direct/risk splits (no hardcoded populations)

import pandas as pd

cy = country_year.copy()

# Computed columns
cy["total_direct_deaths"] = cy["deaths_direct_drug"] + cy["deaths_direct_alcohol"]
cy["total_risk_deaths"]   = cy[["deaths_risk_tobacco", "deaths_risk_drug", "deaths_risk_alcohol"]].sum(axis=1)
cy["total_deaths"]        = cy["total_direct_deaths"] + cy["total_risk_deaths"]
cy["data_type"]           = "Actual"
cy["decade"]              = (cy["year"] // 10) * 10
cy["decade_label"]        = cy["decade"].astype(str) + "s"

# Year-over-year change per country (total_deaths)
cy = cy.sort_values(["country", "year"])
cy["total_deaths_yoy_pct"] = (cy.groupby("country")["total_deaths"]
                                  .pct_change() * 100).round(2)

tableau_dir = PROJECT_DIR / "tableau_ready"
tableau_dir.mkdir(exist_ok=True)

safe_save_csv(cy, tableau_dir / "country_year_enhanced.csv")

# Annual summary with 3yr moving avg + YoY
ann_s = annual.copy()
for col in ALL_M:
    if col in ann_s.columns:
        ann_s[f"{col}_yoy_pct"] = ann_s[col].pct_change() * 100
        ann_s[f"{col}_ma3"]     = ann_s[col].rolling(window=3, center=True).mean()

safe_save_csv(ann_s, tableau_dir / "annual_summary_enhanced.csv")

# Combined global actual + forecast
global_actual = cy.groupby("year").agg(
    deaths_direct_drug   = ("deaths_direct_drug",    "sum"),
    deaths_direct_alcohol= ("deaths_direct_alcohol", "sum"),
    deaths_risk_tobacco  = ("deaths_risk_tobacco",   "sum"),
    deaths_risk_drug     = ("deaths_risk_drug",      "sum"),
    deaths_risk_alcohol  = ("deaths_risk_alcohol",   "sum"),
    total_deaths         = ("total_deaths",           "sum")
).reset_index()
global_actual["data_type"] = "Actual"

forecast_totals                = forecast_wide.copy()
forecast_totals["year"]        = pd.to_datetime(forecast_totals["date"]).dt.year
forecast_totals["total_deaths"]= forecast_totals[MEASURES].sum(axis=1)
forecast_totals["data_type"]   = "Forecast"

combined_global = pd.concat([
    global_actual,
    forecast_totals[["year"] + MEASURES + ["total_deaths", "data_type"]]
], ignore_index=True)

safe_save_csv(combined_global, tableau_dir / "global_actual_forecast.csv")
safe_save_csv(long_df,         tableau_dir / "measures_long.csv")

print("\n✓ All Tableau-ready files saved to:", tableau_dir)
print("Files:")
for f in sorted(tableau_dir.iterdir()):
    print(f"  {f.name}")


In [ ]:
# ── COMMIT 5: Export files (Power BI and Tableau) ────────────────────────────
git_commit_push("feat: Power BI and Tableau export files (Section 10)")


---
## Section 11 — Data Quality Report & Conclusions

Final validation pass before handing off to dashboards.

In [ ]:
# [11] FINAL DATA QUALITY REPORT + KEY FINDINGS

import pandas as pd

cy_check = pd.read_csv(tableau_dir / "country_year_enhanced.csv")

print("=" * 55)
print("  FINAL DATA QUALITY REPORT")
print("=" * 55)

missing = cy_check.isnull().sum()
missing = missing[missing > 0]
if len(missing):
    print("\n⚠ Missing values:")
    display(missing)
else:
    print("\n  ✓ No missing values in enhanced country table")

print(f"\nCoverage  : {cy_check['year'].min()}–{cy_check['year'].max()}")
print(f"Countries : {cy_check['country'].nunique()}")
print(f"Rows      : {len(cy_check):,}")

print("\n" + "=" * 55)
print("  KEY FINDINGS")
print("=" * 55)

top5_global = (cy_check[cy_check["year"] == cy_check["year"].max()]
               .nlargest(5, "total_deaths")[["country", "total_deaths", "continent"]])

print(f"\nTop 5 countries by total substance deaths ({cy_check['year'].max()}):")
display(top5_global.reset_index(drop=True))

# Global change 1990 → last year
first_yr = int(cy_check["year"].min())
last_yr  = int(cy_check["year"].max())
g1 = cy_check[cy_check["year"] == first_yr]["total_deaths"].sum()
g2 = cy_check[cy_check["year"] == last_yr]["total_deaths"].sum()
pct_chg = (g2 - g1) / g1 * 100

print(f"\nGlobal total deaths {first_yr}: {g1:,.0f}")
print(f"Global total deaths {last_yr} : {g2:,.0f}")
print(f"Change over period  : {'↑' if pct_chg > 0 else '↓'} {abs(pct_chg):.1f}%")

print("\n" + "=" * 55)
print("  ANALYSIS COMPLETE ✓")
print("=" * 55)


In [ ]:
# ── COMMIT 6: Quality report and conclusions ──────────────────────────────────
git_commit_push("feat: final data quality report and key findings (Section 11)")


---
## 📋 README Push

This cell writes a professional README to the repo directory and pushes it as a final commit.

In [ ]:
# [12] WRITE README.md TO REPO + FINAL PUSH

readme_content = """# 🧪 Global Substance Abuse Mortality Analytics

> **Portfolio Project** | Eden Mandate AI_Tools  
> Master's in AI and Data Science

## Overview
End-to-end analysis of global mortality attributable to substance abuse (drug, alcohol, tobacco)  
across 207 countries from 1990–2019, with a 5-year SARIMAX forecast through 2024.

## Dataset
- **Source:** Global Burden of Disease / Our World in Data  
- **Shape:** 6,210 rows × 8 columns  
- **Coverage:** 207 countries · 30 years (1990–2019)  
- **Measures:** 5 death categories (direct drug, direct alcohol, risk tobacco, risk drug, risk alcohol)

## Methodology

| Step | Description |
|---|---|
| Data Quality | Null checks, duplicate detection, negative value validation |
| Standardisation | Snake_case normalisation, date parsing, numeric coercion |
| Feature Engineering | Lag features (t-1, t-2), 3-year rolling mean, trend index |
| Modelling | SARIMAX (AIC grid), Prophet (annual-corrected), AutoTS (fast ensemble) |
| Evaluation | Hold-out test set (3 years) · MAE, RMSE, MAPE |
| Forecasting | 5-year horizon per measure with 80% confidence intervals |
| Exports | Power BI and Tableau ready CSVs |

## Key Results

See `model_evaluation.csv` for full metrics. Best model selected by lowest RMSE on the 3-year hold-out.

## Output Files

```
├── annual_totals.csv               # Global annual totals
├── country_year.csv                # Country × year breakdowns
├── measures_long.csv               # Tidy long format
├── fact_actual_forecast.csv        # Actuals + forecasts combined
├── forecast_by_measure_long.csv    # Forecasts in long format
├── forecast_by_measure_wide.csv    # Forecasts in wide format
├── kpi_all.csv                     # KPI summary table
├── model_evaluation.csv            # Model comparison (MAE/RMSE/MAPE)
├── model_report.json               # Full model selection report
└── tableau_ready/                  # Tableau-optimised exports
```

## Tech Stack
Python · pandas · statsmodels (SARIMAX) · Prophet · AutoTS · matplotlib · Google Colab

## Commit Structure
| Commit | Content |
|---|---|
| 1 | Data loading, quality checks, standardisation |
| 2 | KPIs and EDA visualisations |
| 3 | Feature engineering |
| 4 | Modelling, evaluation, forecasting |
| 5 | Export files |
| 6 | Quality report and findings |
| 7 | README |

---
*Built with integrity — real metrics, honest model reporting.*
"""

readme_path = Path(REPO_DIR) / "README.md"
readme_path.write_text(readme_content)
print("✓ README.md written")

git_commit_push("docs: add professional README with methodology and output index")
